# This is the jupyter Notebook which will do the final data-analysis. It will do the tasks necessary for Phase 1:

## 1) Give all Trimble readings (ID_track) a score. According to GPS_classification_15.11.txt

## 2) Pick out the best reading (ID_track) to use where multiple readings are available for the same plant ID.

### Input: The data tables for every year from Phase 0/ November/ Version 3

### Output: 
1) The original data tables with the score for each ID_track

2) A data table that containts the best reading (ID_track) for every Sample


@ Author: Harald Ringbauer

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cPickle as pickle

In [13]:
# A few global Variable names.
# String of the Names of all Input Files:
file_names = ["Antspec2009_IDtrack_DF.csv","Antspec2010_IDtrack_DF.csv","Antspec2011_IDtrack_DF.csv",
              "Antspec2012_IDtrack_DF.csv","Antspec2013_IDtrack_DF.csv","Antspec2014_IDtrack_DF.csv"] 

output_file_name = "Antspec_compl.csv"

input_folder =  "./Input_Data/"    # Name of the Folder used for Input Files
output_folder = "./Output_Data/"

In [14]:
data_tables = []   # List into which all data_tables will go.
all_data = []      # Data Frame for all the Data
headers = []       # List into which all headers will go.


for file_name in file_names:
    path = input_folder + file_name
    print("\nLoading " + path)
    data = np.genfromtxt(path, delimiter = '$', dtype=None)
    print("Shape of Data: ")
    print(data.shape)
    df = pd.read_csv(input_folder + file_name, delimiter="$")     # Create Panda Data Frame
    data_tables.append(df)                                        # Append Panda Data 
    headers.append(list(df))
    
print("\n%i Files successfully loaded!" % len(data_tables))      # Quick Status Update

# Quickly check whether all headers are the same:
for i in range(1,len(headers)):
    equal=np.array_equal(headers[i-1], headers[i])
    if equal == False:
        raise ValueError("Not all headers are equal!")
print("All headers equal!")

print("\nData Format: ")
df = data_tables[0]
print(df.dtypes)

# Concatenate all the data into a single data Frame
print("\nMerging all the data!")
all_data=pd.concat(data_tables, axis=0, ignore_index=1) # Concatenate all Data; Ignore Indexing
print("Nr. of all individuals: %i" % all_data.shape[0])
all_data.reset_index(drop=1)
all_data.reset_index()
all_data.head(3)
#all_data


Loading ./Input_Data/Antspec2009_IDtrack_DF.csv
Shape of Data: 
(1925, 27)

Loading ./Input_Data/Antspec2010_IDtrack_DF.csv
Shape of Data: 
(3119, 27)

Loading ./Input_Data/Antspec2011_IDtrack_DF.csv
Shape of Data: 
(2569, 27)

Loading ./Input_Data/Antspec2012_IDtrack_DF.csv
Shape of Data: 
(3741, 27)

Loading ./Input_Data/Antspec2013_IDtrack_DF.csv
Shape of Data: 
(5758, 27)

Loading ./Input_Data/Antspec2014_IDtrack_DF.csv
Shape of Data: 
(8715, 27)

6 Files successfully loaded!
All headers equal!

Data Format: 
Population                object
Location                  object
VisitID                    int64
PlantID                   object
IDtrack                   object
PhenoCat                 float64
Date                      object
year                       int64
ThisYearLabel              int64
Latitude                 float64
Longitude                float64
Altitude                 float64
CorrectedLatitude        float64
CorrectedLongitude       float64
CorrectedAltitude 

,Population,Location,VisitID,PlantID,IDtrack,PhenoCat,Date,year,ThisYearLabel,Latitude,...,DistEastofCentre,DistNorthOfCentre,HorzErr,VertErr,GPS,Comments,Alive,Perennial,FloweringStage,LocationManualUpdater
0,HZ,Unk,2022.0,J0001,1_J0001,NaN,16/06/09,2009,1.0,42.321497,...,-232.275681,-131.158978,5.95776,9.13216,1.0,NaN,Alive,NaN,NaN,NaN
1,HZ,Unk,2023.0,J0002,2_J0002,NaN,16/06/09,2009,1.0,42.321479,...,-233.310219,-133.051083,5.92379,8.82668,1.0,NaN,Alive,NaN,NaN,NaN
2,HZ,Unk,2024.0,J0003,3_J0003,NaN,16/06/09,2009,1.0,42.321529,...,-241.008310,-127.430149,7.18771,9.08730,1.0,NaN,Alive,NaN,NaN,NaN


In [22]:
#### Now do the ID - Track Score for Every single individual:
all_data['Track_Score']=-1 # Set the Default Track Score to 0!
un_corr_df = all_data.loc[:,['Latitude','Longitude','Altitude']]                          # The uncorrected Data
corr_df = all_data.loc[:,['CorrectedLatitude','CorrectedLongitude','CorrectedAltitude']]  # The corrected Data                                  # The corrected Data

# Extract the indices where Nan
nan_corr = corr_df.isnull().any(axis=1)   # Get all indices were at least one entry is not 0.
nan_uncorr = un_corr_df.isnull().any(axis=1)   # Get all indices were at least one entry is not 0.

# Some logical magic to find out where one entry is 
both_nan = nan_corr & nan_uncorr
one_nan = ~nan_corr & nan_uncorr   # Corrected Value is not NaN, but uncorrected is!
non_nan = ~nan_corr & ~nan_uncorr  # Both are not NaN


# Give some Output
print("Nr. of NaNs in uncorrected: %i" % np.sum(list(nan_uncorr)))
print("Nr. of NaNs in corrected: %i" % np.sum(list(nan_corr)))
print("Nr. of NaNs in both: %i" % np.sum(list(both_nan)))

# Extract where the two Position-Frames differ:
diff_pos = (un_corr_df.values != corr_df.values).any(axis=1)  # Numpy Array where GPS positions different
same_pos = np.logical_not(diff_pos)
print("Nr. Entries where GPS Entries are different: %i out of %i" % (np.sum(list(diff_pos)), corr_df.shape[0]))

bd = non_nan & diff_pos
bs = non_nan & same_pos

# Extract where someone did a manual Update:
manual_updater = ~all_data.LocationManualUpdater.isnull()   # Extract all entries where there is a manual Updater
print("Nr. of Entries with manual positon update:  %i out of %i" % (np.sum(list(manual_updater)), corr_df.shape[0]))

# Extract where death or label only:
plant_stat_wrong = (all_data.Alive == "Label only") | (all_data.Alive == "Dead")
print("Nr. of Entries with Death or Label Only:  %i out of %i" % (np.sum(list(plant_stat_wrong)), corr_df.shape[0]))

# Do the classification per year
first_chars = all_data.PlantID.str[0].values # First Character for every year
years = all_data.year.values      # Year 
#print(set(first_char))  # The C comes from Cadi
#print(set(year))        # The year

char = ['J', 'K'  , 'L' ,'M' , 'P', 'S', 'C']  # Start with creating a dictionary:
year = [2009, 2010, 2011, 2012, 2013, 2014, 2014]
d = dict(zip(char, year))

transf_year = np.array([d[i] for i in first_chars])  # Put first chars through dictionary

same_year = (transf_year == years)
same_year[first_chars=='C']= 1  # Set all Cadi Plants to this year

print("\nPlants from the same year %i:" %np.sum(same_year))
print("Plants from the same year according to label: %i" %np.sum(all_data.ThisYearLabel))

diff = (same_year != all_data.ThisYearLabel.values)
#print(all_data.loc[diff,'PlantID']) # Print Differences between inferred difference and ThisYearLabel.values




############################ Actually set the entries
#print(np.sum(list(one_nan)))
all_data.loc[both_nan,'Track_Score'] = 0  # Set the lowest Score
all_data.loc[one_nan,'Track_Score'] = 1   # Set Score = 1: Corrected is not NaN, but Uncorrected is!
all_data.loc[bs,'Track_Score'] = 2        # Both Entries exist - but are the same
all_data.loc[bd,'Track_Score'] = 3        # High score: Both GPS entries exist and are different

all_data.loc[manual_updater, 'Track_Score'] = 1 # Set all entries where there was an manual Updater to 1.
all_data.loc[plant_stat_wrong, 'Track_Score'] = 0 # Set all entrie where the plant status is wrong to 0.

all_data.to_csv(output_folder + output_file_name, sep = '$')

Nr. of NaNs in uncorrected: 1792
Nr. of NaNs in corrected: 1466
Nr. of NaNs in both: 1466
Nr. Entries where GPS Entries are different: 25690 out of 25821
Nr. of Entries with manual positon update:  404 out of 25821
Nr. of Entries with Death or Label Only:  3225 out of 25821

Plants from the same year 20911:
Plants from the same year according to label: 20817


## Step 2 of Phase 1.
Here we will do a script for the ID Track decision.

In [102]:
# First load the Data Frame
df = pd.read_csv(output_folder + output_file_name, sep="$")     # Create Panda Data Frame
print("Successfully loaded")
print("Shape of Data: ")
print(df.shape)

# First do some statistics to get an overview over the Problem:
counts=df['PlantID'].value_counts()
max_counts = max(counts)
print(counts[:5])
bins = np.arange(-0.5,max_counts+0.5,1)

plt.figure()
plt.hist(counts, bins=bins)
plt.xlim([0,max_counts+1])
plt.xlabel("Nr of Appearance of Label")
#plt.show()

# Now extract unique ID-Labels:
unique_labels=df.PlantID.unique()
unique_labels=np.sort(unique_labels)   # Sort; so that everything is easy to find.
print("Nr of unique Labels: %i" % len(unique_labels))
print("Nr of total Labels: %i" % len(df))

# Create a Data Frame; every unique Label
dic={'PlantID': unique_labels, 'Latitude': 0, 'Longitude': 0, 'Altitude': 0, 'ID_Track Decision': 'None', 'Score': -10}
# At the end; there should be no -10 at the Score Column!!
final_decision = pd.DataFrame(dic, columns=['PlantID','Latitude','Longitude','Altitude','ID_Track Decision',  'Score'])
final_decision.head(10)

Successfully loaded
Shape of Data: 
(25821, 29)
J1548    8
L1359    8
J1251    8
P0056    8
M0121    7
Name: PlantID, dtype: int64
Nr of unique Labels: 18215
Nr of total Labels: 25821


,PlantID,Latitude,Longitude,Altitude,ID_Track Decision,Score
0,CADI0001,0,0,0,None,-10
1,CADI0002,0,0,0,None,-10
2,CADI0003,0,0,0,None,-10
3,CADI0004,0,0,0,None,-10
4,CADI0005,0,0,0,None,-10
5,CADI0006,0,0,0,None,-10
6,CADI0007,0,0,0,None,-10
7,CADI0008,0,0,0,None,-10
8,CADI0009,0,0,0,None,-10
9,CADI0010,0,0,0,None,-10


In [146]:
#Now write functions to set the various Values of the Data Frame:
# Whole date frame is df
def set_score(plantID):
    '''Function to set the score for plant ID'''
    lookup_list = df.PlantID.values   # Define the lookup List
    assert(isinstance(plantID, str))  # To make sure plantID is actually String
    rel_indices = np.where(lookup_list == plantID)[0]
    
    if len(rel_indices) == 0:        # In case Index is not there:
        print("Index not found!!")
        print("Put all your shit in bag. But get it together!")
        return
    
    matching_plants = df.iloc[rel_indices,:]    # Where to find the plants with the right PlantID; then extract them
    coords_matching = matching_plants.loc[:,['CorrectedLatitude','CorrectedLongitude','CorrectedAltitude']] 
    
    ############################################
    # Set Zero Score Plants: All GPS Entries NaN
    missing=coords_matching.isnull().any(axis=1)
    
    if np.sum(~missing)==0:  # If only missing Values
        return 0   # Return Score 0!
    
    ############################################
    # Set Score to One:
    
    if np.sum(~missing)
    
    
        
        
    print(missing)
    return coords_matching
    
set_score("J1548")

5
1571     False
2514      True
2515     False
5175     False
7697     False
7698     False
11527     True
25653     True
dtype: bool


,CorrectedLatitude,CorrectedLongitude,CorrectedAltitude
1571,42.323150,2.076300,1147.00
2514,NaN,NaN,NaN
2515,42.322972,2.075709,1142.41
5175,42.322952,2.075704,1141.62
7697,42.322938,2.075665,1142.63
7698,42.322955,2.075683,1144.07
11527,NaN,NaN,NaN
25653,NaN,NaN,NaN


## Test Area
Created to play around with and try out stuff.

In [11]:
plant_ID=all_data.PlantID
plant_ID.str[0]

0        J
1        J
2        J
3        J
4        J
5        J
6        J
7        J
8        J
9        J
10       J
11       J
12       J
13       J
14       J
15       J
16       J
17       J
18       J
19       J
20       J
21       J
22       J
23       J
24       J
25       J
26       J
27       J
28       J
29       J
        ..
25791    P
25792    P
25793    P
25794    P
25795    P
25796    P
25797    P
25798    P
25799    P
25800    P
25801    P
25802    P
25803    P
25804    P
25805    P
25806    P
25807    P
25808    P
25809    P
25810    P
25811    P
25812    P
25813    P
25814    P
25815    P
25816    P
25817    P
25818    P
25819    P
25820    P
Name: PlantID, dtype: object

In [79]:
# A test data frame created for playing around:

test_df = pd.DataFrame({ 'A' : 1., 'B' : pd.Timestamp('20130102'), 'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                                      'D' : np.array([3] * 4,dtype='int32'),
                                      'Ea' : pd.Categorical(["test","train","test","train"]),
                                      'Fe' : 'foo' })

In [80]:
test_df

,A,B,C,D,Ea,Fe
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


In [140]:
test_df['G']=0
print(test_df.dtypes)
test_df.loc[:,['A','Ea','G']]
test_df.iloc[2, 3] = np.nan
test_df
test_df.isnull().any(axis=1)

A            float64
B     datetime64[ns]
C            float32
D            float64
Ea          category
Fe            object
G              int64
dtype: object


[False, False, True, False]

In [181]:
test_df = pd.DataFrame({ 'A' : 1., 'B' : pd.Timestamp('20130102'), 'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                                      'D' : np.array([3] * 4,dtype='int32'),
                                      'Ea' : pd.Categorical(["test","train","test","train"]),
                                      'Fe' : 'foo' })
test_df1 = pd.DataFrame({ 'A' : 1, 'B' : pd.Timestamp('20130102'), 'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                                      'D' : np.array([3] * 4,dtype='int32'),
                                      'Ea' : pd.Categorical(["test","train","test","train"]),
                                      'Fe' : 'foo' })
test_df1.iloc[2,3]=2.0
test_df1

a=(test_df.values!=test_df1.values).any(axis=1)
a

array([False, False,  True, False], dtype=bool)

In [6]:
a = np.array([1,2,3])
b = np.array([1,2,4])
a == b
print("Test2")

Test2


In [31]:
a = pd.DataFrame({"Test": [1,2,2,3,3,3,3]})
a["Test"].value_counts()

3    4
2    2
1    1
Name: Test, dtype: int64